In [1]:
import pandas as pd
import os
import numpy as np
from math import sqrt

from sklearn.linear_model import Lasso, ElasticNet, Ridge, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer, explained_variance_score, mean_absolute_error
from sklearn.metrics import median_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from nltk.stem.snowball import SnowballStemmer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV

In [12]:
def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))

def mse(y, y_pred):
    return mean_squared_error(y, y_pred)

def exp_var(y, y_pred):
    return explained_variance_score(y, y_pred)

def mae(y, y_pred):
    return mean_absolute_error(y, y_pred)

def r2(y, y_pred):
    return r2_score(y, y_pred)

def medae(y, y_pred):
    return median_absolute_error(y, y_pred)

depths = [3,4,5,6,7,8, 9,10,11,12,13,14,15]
ests = [1,2,3,4,5,6,7,8, 9,10,11,12,13,14,15]
feats = [1,2,3,4,5,6,7,8, 9,10]

regressors = {
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'kNN': KNeighborsRegressor(),
    'SVRrbf': SVR(kernel='rbf', gamma='scale'),
    'SVRpoly': SVR(kernel='poly', gamma='scale'),
    'SVRlinear': SVR(kernel='linear', gamma='scale'),
    'DT': DecisionTreeRegressor(max_depth=5),#GridSearchCV(estimator=DecisionTreeRegressor(), param_grid={'max_depth': depths}),
    'RF': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=2),#GridSearchCV(estimator=RandomForestRegressor(), param_grid={'max_depth': depths, 'n_estimators': ests, 'max_features': feats}),
    'MLP': MLPRegressor(hidden_layer_sizes=(200,50), max_iter=2000),
    'AdaBoost': AdaBoostRegressor(n_estimators=10),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=10, max_depth= 5)
}

In [3]:
from tqdm import tqdm_notebook
import IPython.display as ipd

results = dict()
def cross_val_regression(regressors, features, labels, preprocessfunc):
    columns = list(regressors.keys())
    scores = pd.DataFrame(columns=columns, index=['RMSE', 'MSE', 'MAE', 'Median AE'])
    #, 'Explained Variance', 'R2 Score'

    for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):
        scorer = {'rmse': make_scorer(rmse), 'mse':make_scorer(mse), 'mae':make_scorer(mae), 'medae': make_scorer(medae), 'expvar': make_scorer(exp_var), 'r2': make_scorer(r2)}
        reg = make_pipeline(*preprocessfunc, reg)
        reg_score = cross_validate(reg, features, labels, scoring=scorer, cv=10, return_train_score=True, return_estimator=True) 
        scores.loc['RMSE', reg_name] = reg_score['test_rmse'].mean()
        scores.loc['MSE', reg_name] = reg_score['test_mse'].mean()
        scores.loc['MAE', reg_name] = reg_score['test_mae'].mean()
        scores.loc['Median AE', reg_name] = reg_score['test_medae'].mean()
        #scores.loc['Explained Variance', reg_name] = max(reg_score['test_expvar'])
        #scores.loc['R2 Score', reg_name] = max(reg_score['test_r2'])
        if reg_name not in results:
            results[reg_name] = list()
        else:
            results[reg_name].append(reg_score)
            
    #mean_rmse = scores.loc['RMSE', :].mean()
    #std_rmse = scores.loc['RMSE', :].std()
    #mean_mse = scores.loc['MSE', :].mean()
    #std_mse = scores.loc['MSE', :].std()
    
    #scores.loc['RMSE']['Mean'] = mean_rmse
    #scores.loc['RMSE']['std'] = std_rmse
    #scores.loc['MSE']['Mean'] = mean_mse
    #scores.loc['MSE']['std'] = std_mse
    return scores

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s)
        is_min = s == min(s)
        is_max_or_min = (is_min | is_max)
        #if s.index[0]=='Explained Variance' or s.index[0]=='R2 Score':
            #return ['background-color: yellow' if v else '' for v in is_max]
        #else:
        return ['background-color: cyan' if v else '' for v in is_min]
    scores = scores.style.apply(highlight, axis=1, subset=pd.IndexSlice[:, :scores.columns[-2]])
    return scores.format('{:.8f}')

In [13]:
tf_idf = pd.read_csv('tfidf_PCA.csv')
tf_idf.drop('Unnamed: 0', axis=1, inplace=True)
tf_idf

,0,1,2,3,4,5,6,7,8,9,...,566,567,568,569,570,571,572,573,574,575
0,0.372514,-1.131089,-4.053200,0.036860,0.433837,4.066642,-0.411591,-0.697551,13.444840,8.207451,...,-0.057821,-0.067388,0.039689,0.022218,-0.032597,0.083274,-0.034556,-0.006326,-0.081149,0.023123
1,-0.021232,-0.068678,-2.645587,0.336421,0.208905,1.439549,-0.206336,-0.253360,2.568004,3.105128,...,-0.044444,0.111993,-0.156471,0.163564,0.062939,-0.112892,0.191584,0.043665,-0.075168,0.052911
2,-0.437151,-0.525415,-0.664850,-0.210955,-0.064710,-0.566562,-0.203647,-0.079869,0.723133,0.203518,...,-0.025932,0.039786,0.059235,-0.234305,-0.015047,0.099914,-0.052799,-0.167495,0.187817,-0.060391
3,0.302196,-0.833174,-1.614257,0.596998,0.106135,-0.987560,-0.259543,-0.100091,-0.150844,0.354404,...,0.013733,0.009456,-0.046425,0.006060,-0.006678,0.044585,0.016278,0.047413,-0.127037,0.051024
4,-0.484528,-0.380687,-1.162074,-0.398932,0.094664,1.049898,1.490700,-0.071681,0.639735,0.061694,...,0.143784,0.011110,0.027412,0.041198,0.007037,-0.039545,0.013504,0.023842,0.030525,0.006152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,-0.648877,-0.473254,-0.666935,-0.839443,0.045920,-0.865274,-0.240840,-0.004606,-0.612131,-0.886945,...,-0.021916,-0.014778,0.323793,-0.016760,-1.089046,0.120146,-0.034135,0.357434,-0.094611,-0.050540
619,-0.811595,-0.366282,-0.406003,-0.871507,0.017761,-0.963611,-0.106265,-0.001111,-0.646797,-0.969973,...,-1.723127,-2.133114,1.203167,4.158603,-1.694081,-2.871828,-0.800811,3.081932,-2.928625,-1.130741
620,-0.812087,-0.114974,-0.792614,-0.837195,0.060975,-0.885524,-0.173591,-0.004911,-0.779217,-0.746024,...,0.027397,0.136398,-0.232210,0.221880,-0.018520,0.490769,0.504708,-0.687689,0.248730,0.083299
621,-0.824177,-0.450140,-0.659544,-0.918170,0.048541,-1.016463,-0.105731,-0.001834,-0.700440,-0.627355,...,-0.321950,2.026346,-0.287379,0.061485,0.958677,1.182823,0.479887,-0.867533,1.958348,-0.181946


In [14]:
linguistic_features = pd.read_csv('linguistic_features_02.csv')
linguistic_features.drop('Unnamed: 0', axis=1, inplace=True)
linguistic_features

,musicId,positive,negative,neutral,compound,Polarity,Introspection,Temper,Attitude,Sensitivity,...,Avg. Chorus Word Length,Song Vocabulary Richness,Chorus Vocabulary Richness,Function Word Frequencies,DocVec Norm,InferVec Norm,Arousal(mean),Valence(mean),Arousal(std),Valence(std)
0,1,0.299,0.101,0.600,0.9966,0.039695,0.026058,0.000018,0.016646,0.029991,...,3.736842,0.019875,0.091241,0.001442,2.245172,0.201296,0.4000,0.5750,0.156125,0.160078
1,5,0.096,0.338,0.567,-0.9984,0.070948,0.076279,0.005977,0.013302,0.023026,...,4.064516,0.015248,0.024412,0.001506,1.793216,0.016271,0.1500,0.2000,0.175000,0.203101
2,6,0.038,0.315,0.647,-0.9988,-0.018859,-0.012290,-0.001244,-0.000046,0.045332,...,5.125000,0.017640,0.350000,0.001190,2.791167,0.017019,0.5125,0.3500,0.171847,0.200000
3,9,0.132,0.232,0.636,-0.9879,-0.030606,0.001858,-0.006965,0.013221,-0.047465,...,3.835443,0.023221,0.083636,0.001210,1.114908,0.076152,0.4500,0.2875,0.203101,0.158607
4,10,0.097,0.197,0.706,-0.9909,0.009457,0.059946,-0.012350,0.005107,0.010768,...,4.044944,0.015464,0.076412,0.001000,1.566652,0.441977,0.4375,0.4250,0.195656,0.139194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,983,0.326,0.136,0.537,0.9913,-0.012644,-0.038148,-0.030269,0.051171,-0.051537,...,3.250000,0.019688,0.800000,0.001164,1.363090,0.016947,0.7750,0.5000,0.108972,0.158114
599,984,0.365,0.045,0.590,0.9984,0.069699,0.072205,-0.046917,0.021747,0.039524,...,3.111111,0.022202,0.625000,0.001418,1.647133,0.016490,0.8250,0.7375,0.100000,0.087500
600,985,0.137,0.065,0.798,0.9552,0.054464,0.018446,0.033794,0.029519,-0.038880,...,2.600000,0.018182,0.769231,0.001352,1.471850,0.016133,0.7500,0.7000,0.147902,0.160078
601,993,0.041,0.056,0.904,-0.2168,0.074153,0.042398,0.030127,0.034907,0.034610,...,2.666667,0.032670,0.000000,0.001468,1.066196,0.016926,0.8625,0.7625,0.103833,0.152582


In [15]:
lf = pd.read_csv('linguistic_features_01.csv')
lf.drop('Unnamed: 0', axis=1, inplace=True)
index = lf.loc[pd.isna(lf["Arousal(mean)"]), :].index
tf_idf.drop(index, inplace=True)
tf_idf

,0,1,2,3,4,5,6,7,8,9,...,566,567,568,569,570,571,572,573,574,575
0,0.372514,-1.131089,-4.053200,0.036860,0.433837,4.066642,-0.411591,-0.697551,13.444840,8.207451,...,-0.057821,-0.067388,0.039689,0.022218,-0.032597,0.083274,-0.034556,-0.006326,-0.081149,0.023123
1,-0.021232,-0.068678,-2.645587,0.336421,0.208905,1.439549,-0.206336,-0.253360,2.568004,3.105128,...,-0.044444,0.111993,-0.156471,0.163564,0.062939,-0.112892,0.191584,0.043665,-0.075168,0.052911
2,-0.437151,-0.525415,-0.664850,-0.210955,-0.064710,-0.566562,-0.203647,-0.079869,0.723133,0.203518,...,-0.025932,0.039786,0.059235,-0.234305,-0.015047,0.099914,-0.052799,-0.167495,0.187817,-0.060391
3,0.302196,-0.833174,-1.614257,0.596998,0.106135,-0.987560,-0.259543,-0.100091,-0.150844,0.354404,...,0.013733,0.009456,-0.046425,0.006060,-0.006678,0.044585,0.016278,0.047413,-0.127037,0.051024
4,-0.484528,-0.380687,-1.162074,-0.398932,0.094664,1.049898,1.490700,-0.071681,0.639735,0.061694,...,0.143784,0.011110,0.027412,0.041198,0.007037,-0.039545,0.013504,0.023842,0.030525,0.006152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,-0.648877,-0.473254,-0.666935,-0.839443,0.045920,-0.865274,-0.240840,-0.004606,-0.612131,-0.886945,...,-0.021916,-0.014778,0.323793,-0.016760,-1.089046,0.120146,-0.034135,0.357434,-0.094611,-0.050540
619,-0.811595,-0.366282,-0.406003,-0.871507,0.017761,-0.963611,-0.106265,-0.001111,-0.646797,-0.969973,...,-1.723127,-2.133114,1.203167,4.158603,-1.694081,-2.871828,-0.800811,3.081932,-2.928625,-1.130741
620,-0.812087,-0.114974,-0.792614,-0.837195,0.060975,-0.885524,-0.173591,-0.004911,-0.779217,-0.746024,...,0.027397,0.136398,-0.232210,0.221880,-0.018520,0.490769,0.504708,-0.687689,0.248730,0.083299
621,-0.824177,-0.450140,-0.659544,-0.918170,0.048541,-1.016463,-0.105731,-0.001834,-0.700440,-0.627355,...,-0.321950,2.026346,-0.287379,0.061485,0.958677,1.182823,0.479887,-0.867533,1.958348,-0.181946


In [16]:
X = tf_idf
prefunc = [StandardScaler()]

print('In Arousal dimension...')
y_a = linguistic_features['Arousal(mean)']
scores_l_a = cross_val_regression(regressors, X, y_a, prefunc)
ipd.display(format_scores(scores_l_a))

print('In Valence dimension...')
y_v = linguistic_features['Valence(mean)']
scores_l_v = cross_val_regression(regressors, X, y_v, prefunc)
ipd.display(format_scores(scores_l_v))

In Arousal dimension...


<ipython-input-3-730f3bd8eff5>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.18352704,1.07906981,0.18352704,0.26340231,0.18299937,0.21311892,0.72076323,0.20502158,0.17956206,0.80692547,0.17959692,0.17635807
MSE,0.03391291,1.17330822,0.03391291,0.07001977,0.03363920,0.05097320,0.54531751,0.04227979,0.03239679,0.67388715,0.03248586,0.03126479
MAE,0.15258288,0.80993766,0.15258288,0.20796412,0.15236205,0.15749068,0.52640600,0.16278845,0.14538966,0.60426289,0.14499870,0.14361649
Median AE,0.13923210,0.62012283,0.13923210,0.16251136,0.13457425,0.13356401,0.39122182,0.13855236,0.12566616,0.44770357,0.12145157,0.12281364


In Valence dimension...


<ipython-input-3-730f3bd8eff5>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.16049876,0.84035775,0.16049876,0.25395067,0.15664895,0.21046890,0.36422257,0.16442811,0.14841693,0.74426154,0.14724718,0.14510114
MSE,0.02591777,0.71258586,0.02591777,0.06503739,0.02464774,0.05301206,0.13599441,0.02723655,0.02213927,0.56778619,0.02185911,0.02122317
MAE,0.13368844,0.61961053,0.13368844,0.20470771,0.13175670,0.14270130,0.25644141,0.13045882,0.12137191,0.56251181,0.11883138,0.11895406
Median AE,0.12279371,0.46730985,0.12279371,0.17600000,0.12444092,0.11818822,0.18429415,0.10806097,0.10808079,0.44242995,0.10556753,0.10665177


In [17]:
scores_l_a.to_csv('Lyrics-Arousal-mean-tfidf-nogs.csv')
scores_l_v.to_csv('Lyrics-Valence-mean-tfidf-nogs.csv')

In [18]:
y_a = linguistic_features['Arousal(mean)']
y_v = linguistic_features['Valence(mean)']
linguistic_features.drop('Arousal(mean)', axis=1, inplace=True)
linguistic_features.drop('Valence(mean)', axis=1, inplace=True)
linguistic_features.drop('Arousal(std)', axis=1, inplace=True)
linguistic_features.drop('Valence(std)', axis=1, inplace=True)
X = linguistic_features

In [20]:
prefunc = [StandardScaler()]

print('In Arousal dimension...')
scores_l_a = cross_val_regression(regressors, X, y_a, prefunc)
ipd.display(format_scores(scores_l_a))

print('In Valence dimension...')
scores_l_v = cross_val_regression(regressors, X, y_v, prefunc)
ipd.display(format_scores(scores_l_v))

In Arousal dimension...


<ipython-input-3-730f3bd8eff5>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.18352704,0.18209975,0.18352704,0.19422369,0.18846902,0.31659019,0.18307412,0.20276469,0.18002981,0.23119990,0.18201551,0.17902691
MSE,0.03391291,0.03337366,0.03391291,0.03786341,0.03567546,0.12068950,0.03381070,0.04164905,0.03264410,0.05396081,0.03333800,0.03230825
MAE,0.15258288,0.14855149,0.15258288,0.15871498,0.15384048,0.19408549,0.14671345,0.16226811,0.14762248,0.17968585,0.14978791,0.14590650
Median AE,0.13923210,0.13509105,0.13923210,0.13554545,0.13470428,0.14167030,0.12325538,0.13592512,0.12747856,0.14740273,0.13345523,0.12968558


In Valence dimension...


<ipython-input-3-730f3bd8eff5>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.16049876,0.15493596,0.16049876,0.15959063,0.15534083,0.34761676,0.15434843,0.16606373,0.14626156,0.20615116,0.14839913,0.14930776
MSE,0.02591777,0.02410107,0.02591777,0.02561694,0.02427597,0.18955573,0.02395322,0.02791104,0.02151801,0.04408503,0.02220444,0.02243821
MAE,0.13368844,0.12768050,0.13368844,0.13074616,0.12624660,0.17675654,0.12655785,0.13106783,0.11927080,0.15691414,0.12018221,0.12083204
Median AE,0.12279371,0.11402893,0.12279371,0.10951136,0.10658353,0.12388680,0.11460045,0.10761723,0.10503014,0.12853039,0.10380817,0.10487388


In [21]:
scores_l_a.to_csv('Lyrics-Arousal-mean-linguistic-nogs.csv')
scores_l_v.to_csv('Lyrics-Valence-mean-linguistic-nogs.csv')

In [11]:
all_results['Valence(mean)'] = copy.deepcopy(results)
results = dict()

In [12]:
tf_idf = pd.read_csv('tfidf_PCA.csv')
tf_idf.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
linguistic_features = pd.read_csv('linguistic_features_02.csv')
linguistic_features.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
lf = pd.read_csv('linguistic_features_01.csv')
lf.drop('Unnamed: 0', axis=1, inplace=True)
index = lf.loc[pd.isna(lf["Arousal(mean)"]), :].index
tf_idf.drop(index, inplace=True)

In [15]:
X = tf_idf
prefunc = [StandardScaler()]

print('In Arousal dimension...')
y_a = linguistic_features['Arousal(std)']
scores_l_a = cross_val_regression(regressors, X, y_a, prefunc)
ipd.display(format_scores(scores_l_a))

print('In Valence dimension...')
y_v = linguistic_features['Valence(std)']
scores_l_v = cross_val_regression(regressors, X, y_v, prefunc)
ipd.display(format_scores(scores_l_v))

In Arousal dimension...


<ipython-input-3-a6c252af81ad>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.046,0.286,0.046,0.047,0.051,0.048,0.049,0.052,0.045,0.696,0.045,0.045
MSE,0.002,0.082,0.002,0.002,0.003,0.002,0.002,0.003,0.002,0.516,0.002,0.002
MAE,0.037,0.208,0.037,0.037,0.042,0.040,0.041,0.040,0.036,0.499,0.036,0.036
Median AE,0.031,0.150,0.031,0.031,0.039,0.037,0.037,0.032,0.030,0.392,0.031,0.030


In Valence dimension...


<ipython-input-3-a6c252af81ad>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.045,0.316,0.045,0.048,0.048,0.048,0.049,0.050,0.045,0.597,0.047,0.046
MSE,0.002,0.101,0.002,0.002,0.002,0.002,0.002,0.003,0.002,0.366,0.002,0.002
MAE,0.036,0.224,0.036,0.038,0.040,0.040,0.040,0.039,0.036,0.455,0.037,0.036
Median AE,0.031,0.150,0.031,0.030,0.035,0.036,0.036,0.031,0.031,0.365,0.030,0.031


In [16]:
import copy
all_results = dict()
all_results['Arousal(std)'] = copy.deepcopy(results)
results = dict()

In [17]:
y_a = linguistic_features['Arousal(std)']
y_v = linguistic_features['Valence(std)']
linguistic_features.drop('Arousal(mean)', axis=1, inplace=True)
linguistic_features.drop('Valence(mean)', axis=1, inplace=True)
linguistic_features.drop('Arousal(std)', axis=1, inplace=True)
linguistic_features.drop('Valence(std)', axis=1, inplace=True)
X = linguistic_features

In [18]:
prefunc = [StandardScaler()]

print('In Arousal dimension...')
scores_l_a = cross_val_regression(regressors, X, y_a, prefunc)
ipd.display(format_scores(scores_l_a))

print('In Valence dimension...')
scores_l_v = cross_val_regression(regressors, X, y_v, prefunc)
ipd.display(format_scores(scores_l_v))

In Arousal dimension...


<ipython-input-3-a6c252af81ad>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.046,0.046,0.046,0.048,0.049,0.076,0.053,0.051,0.045,0.110,0.045,0.045
MSE,0.002,0.002,0.002,0.002,0.002,0.008,0.003,0.003,0.002,0.013,0.002,0.002
MAE,0.037,0.036,0.037,0.038,0.041,0.046,0.043,0.040,0.036,0.081,0.037,0.035
Median AE,0.031,0.031,0.031,0.033,0.037,0.039,0.040,0.034,0.030,0.066,0.032,0.030


In Valence dimension...


<ipython-input-3-a6c252af81ad>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for reg_name, reg in tqdm_notebook(regressors.items(), desc='regressors'):


,Lasso,Ridge,ElasticNet,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,GradientBoosting
RMSE,0.045,0.046,0.045,0.048,0.048,0.055,0.052,0.054,0.046,0.103,0.046,0.046
MSE,0.002,0.002,0.002,0.002,0.002,0.003,0.003,0.003,0.002,0.011,0.002,0.002
MAE,0.036,0.037,0.036,0.038,0.040,0.040,0.042,0.042,0.037,0.079,0.036,0.037
Median AE,0.031,0.032,0.031,0.033,0.035,0.032,0.035,0.035,0.032,0.060,0.032,0.032


In [19]:
all_results['Valence(std)'] = copy.deepcopy(results)
results = dict()

In [21]:
all_results.keys()

dict_keys(['Arousal(std)', 'Valence(std)'])